# Training OVERLAECOBO
En este notebook se llevaran acabo las pruebas con OVERLAECOBO, se probaran los distintos ZOOMS y la informacion extra para asi escoger la mejor configuracion de dataset y de red neuronal. Como primer paso sera obtener el repositorio donde estan las redes entrenadas y preparalo para su funcionamiento con OVERLAECOBO

In [ ]:
!git clone https://github.com/GeorgeSeif/Semantic-Segmentation-Suite.git

¿Que pruebas se van a realizar?
* Un entrenamiento con cada uno de los zooms sin informacion aumentada
* Un entrenamiento con cada uno de los zooms con informacion aumentada

Las pruebas tendran la etiqueta del lugar donde fueron realizadas, esto debido a que hoy (24 - 05 -18) aun no tengo una cuenta e guane, sin olvidar que cuando la tenga debo hacer configuraciones e instalar tensorflow lo que conllevara bastante tiempo, tiempo en el que podria hacer pruebas en mi pc. Tambien se documentará los parametros a usar, epoch, batch_size, etc.

## 1. Preparar el dataset para usar la Suite
Dado que la suite tiene su propia estructura para el dataset, se hace necesario cambiar los directorios a dicha estructura, y aqui se presenta un inconveniente y es que segun un [issue](https://github.com/GeorgeSeif/Semantic-Segmentation-Suite/issues/65) que encontre hoy (24 - 05 - 18 7:18 am) y segun eso de hace una hora, en el framework no usan las imagenes de test, entonces:
1.  mi test se vuelve el nuevo valid y no existitian imagenes para test y siguo usando el framework
2. cambio todo lo interno del repo para que use test y duro otra semana sacando mi dataset con la nueva division de 3 partes y no solo dos como lo hice
3. busco otro repositorio

Me quedaré con la segunda opcion, la tercera no me cuadra ya con este esta sirviendo, podria atacar el problema con otras cuestiones, primero el issue es muy reciente y podria estar pendiente de cuando haga los arreglos y pues solo seria volver a copiar el repo y si no dan solucion tan solo crear un metodo para sacar un score, para predecir se predice con una imagen, le paso for a la carpeta test, que predicga y luego trato de sacar algun score haciedno comparaciones alfin y alcabo la cuestion es de que entrene

### 1.1 Restructurando dataset
La suite pide una estrucutra de dataset de la siguiente manera.

           - dataset
               - train
               - train_labels
               - valid
               - valid_labels
               - class_dict.scv

cabe aclarar que esta vendria siendo la estructura para un unico zoom asi que dentro de la carpeta actual de OVERLAECOBO creare nuevas carpetas con cada zoom y luego dentro dividir en train y valid.

In [ ]:
import os

In [ ]:
for i in range(8):
    os.mkdir("./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13))

In [ ]:
for i in range(8):
    os.rename("./data/dataset/OVERLAECOBO/train/satellite/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/train")
    os.rename("./data/dataset/OVERLAECOBO/train/economic_layer/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/train_labels")
    os.rename("./data/dataset/OVERLAECOBO/test/satellite/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/val") 
    os.rename("./data/dataset/OVERLAECOBO/test/economic_layer/"+str(i+13), "./data/dataset/OVERLAECOBO/OVERLAECOBO_"+str(i+13)+"/val_labels")    

Queria copiar con codigo los archivos class_dict pero al parece la libreia os no tiene un `cp` y con shutil habia como 4 metodos para copiar, aisque lo hice manual
### 1.2 Switch Train Test
Dado que al parecer las carpetas valid tienes mas datos de las de train hare un switch entoncs valid pasara a ser train y train valid


In [ ]:
for i in range(7):
    os.rename("./OVERLAECOBO_"+str(i+14)+"/train","./OVERLAECOBO_"+str(i+14)+"/pibote")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/train_labels","./OVERLAECOBO_"+str(i+14)+"/pibote_labels")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/valid","./OVERLAECOBO_"+str(i+14)+"/train")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/valid_labels","./OVERLAECOBO_"+str(i+14)+"/train_labels")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/pibote","./OVERLAECOBO_"+str(i+14)+"/val")
    os.rename("./OVERLAECOBO_"+str(i+14)+"/pibote_labels","./OVERLAECOBO_"+str(i+14)+"/val_labels")   

## 2. Metricas
Al parecer el framework usa varias metricas provenientes de sklearn, y lo se dado a que en uno de sus archivos `.py` se encuentra el siguiente import

      from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
   
y mirando en la documentacion de sklearn, dichas metricas significan [precision_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score), [recall_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score), [f1_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)

## 3. Pruebas
Y por ahora (24 - May - 2018) que aun no tengo la cuenta de guane iniciare las pruebas e mi GPU y con los ZOOMs mas grandes puesto que hay menos imagenes. una de las cosas es que podria probar varios modelos, entre los que permite la suite

* DenseNet56
* FC-DenseNet67
* FC-DenseNet103 
* Encoder-Decoder
* Encoder-Decoder-Skip
* RefineNet-Res50
* RefineNet-Res101 
* RefineNet-Res152
* FRRN-A
* FRRN-B
* MobileUNet
* MobileUNet-Skip
* PSPNet-Res50 
* PSPNet-Res101
* PSPNet-Res152
* GCN-Res50 
* GCN-Res101
* GCN-Res152

Para cada prueba se realizara un train, un test y un predcit. el predict será de una imagen cualquiera del directorio valid

### 3.1 Sin Información extra

Las pruebas a realizar a continuacion son pruebas echas meramente con los 3 canales de una imagen RGB, para las pruebas con informacion extra se agregaran uno o mas canales, dependiendo de la informacion obtenida y la capacidad del framework

## Prueba multi capa

In [3]:
import os
from skimage import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [ ]:
for i in range(7):
    img = io.imread("./OVERLAECOBO_13_multi/train/"+str(i)+".png")
    d = img.flatten()
    f = list(d) + list(d)
    g = np.array(f).reshape((256,256,6))[:,:,:4]
    io.imsave("./OVERLAECOBO_13_multi/train/"+str(i)+".png",g)

In [ ]:
img = io.imread("./OVERLAECOBO_13_multi/train/0.tiff")
img.shape

### 4. Pruebas
Cada una de las pruebas tendra su propio notebook por cada nivel de Zoom

<center><h1 style="color:#B71C1C"> RESULTADOS</h1></center>

Acontinuacion los resultados de las distintas pruebas realizadas.


In [3]:
import os
from skimage import io
import numpy as np
import pandas as pd
import base64
from IPython.display import display, Javascript
from IPython.display import display, HTML
from IPython.utils.py3compat import str_to_bytes, bytes_to_str

In [16]:
ZOOM = "16_COMPOUND"

In [17]:
PATH_PRUEBAS = "./pruebas/OVERLAECOBO_#"
pruebas = pd.read_csv(PATH_PRUEBAS.replace("#",ZOOM)+"/pruebas.csv")
pruebas

,PRUEBA,RED,EPOCH,BATCH_SIZE
0,1,FC-DenseNet56,50,1
1,2,FC-DenseNet56,70,1
2,3,Encoder-Decoder,70,1
3,4,Encoder-Decoder,70,2
4,5,Encoder-Decoder-Skip,70,1
5,6,Encoder-Decoder-Skip,70,2
6,7,FRRN-A,70,1
7,8,MobileUNet,70,1
8,9,MobileUNet,70,2
9,10,MobileUNet-Skip,70,1


In [20]:
PRUEBA = 11

In [21]:
href_train = PATH_PRUEBAS.replace("#",str(ZOOM))+"/prueba_"+str(PRUEBA)+"/train.txt"
href_test = PATH_PRUEBAS.replace("#",str(ZOOM))+"/prueba_"+str(PRUEBA)+"/test.txt"
src_1 = PATH_PRUEBAS.replace("#",str(ZOOM))+"/prueba_"+str(PRUEBA)+"/accuracy_vs_epochs.png"
src_2 = PATH_PRUEBAS.replace("#",str(ZOOM))+"/prueba_"+str(PRUEBA)+"/loss_vs_epochs.png"
src_test = PATH_PRUEBAS.replace("#",str(ZOOM))+"/prueba_"+str(PRUEBA)+"/Val/0_gt.png"

html = """<center><h1> ZOOM = {ZOOM} PRUEBA = {PRUEBA} </h1></center>
          <h3>1. TRAIN FILE</h3></br><a href='{href_train}'>Train file</a>
          <h3>2. SCORE IMAGE</h3>
          <img src='{src_1}'/>
          <h3>3. LOSS IMAGE</h3>
          <img src='{src_2}'/>
          <h3>4. TEST FILE</h3>
          </br><a href='{href_test}'>Test file</a>
          <h3>5. TEST IMAGES</h3>
          <img src='{src_test}'/>
          """
display( HTML(html.format(href_train=href_train,src_1=src_1, src_2=src_2,ZOOM=ZOOM,PRUEBA=PRUEBA,href_test=href_test,src_test=src_test)))
resultados_val =  pd.read_csv(PATH_PRUEBAS.replace("#",ZOOM)+"/prueba_"+str(PRUEBA)+"/Val/val_scores.csv")
resultados_val

,val_name,avg_accuracy,precision,recall,f1 score,mean iou Estrato_1,Estrato_2,Estrato_3,Estrato_4,Estrato_5,Estrato_6,No_Definido
0,0.348465,0.688958,0.348465,0.406254,0.164910,0.0,0.135554,0.426358,1.000000,1.000000,1.000000,0.996776
1,0.437103,0.610767,0.437103,0.470437,0.214110,0.0,0.257552,0.271530,1.000000,1.000000,1.000000,0.918144
10,0.631027,0.789399,0.631027,0.684152,0.245606,0.0,0.278093,0.000000,1.000000,1.000000,1.000000,0.979256
100,0.693665,0.705284,0.693665,0.699226,0.182484,0.0,0.000000,0.004695,1.000000,1.000000,1.000000,0.850411
101,0.287399,0.554186,0.287399,0.371531,0.135867,0.0,1.000000,0.229903,0.003460,1.000000,1.000000,0.915666
102,0.067810,0.766587,0.067810,0.113744,0.028137,0.0,1.000000,0.067523,0.001708,1.000000,1.000000,0.957552
103,0.137634,0.673613,0.137634,0.082106,0.041708,0.0,0.000000,0.249875,0.003067,0.011673,1.000000,0.941532
104,0.267609,0.602539,0.267609,0.312751,0.089819,0.0,0.020619,0.259940,1.000000,0.000000,1.000000,0.904817
105,0.511215,0.713772,0.511215,0.595641,0.266660,0.0,0.300324,1.000000,1.000000,1.000000,1.000000,0.965121
106,0.042953,0.685491,0.042953,0.080173,0.014466,0.0,1.000000,0.117376,0.000389,0.000000,1.000000,0.986175


In [15]:
from skimage import io
img =  io.imread("../data/dataset/OVERLAECOBO/OVERLAECOBO_15/LABEL/train_labels/0.png")

In [16]:
img

array([[[192, 192, 192],
        [192, 192, 192],
        [192, 192, 192],
        ...,
        [192, 192, 192],
        [192, 192, 192],
        [192, 192, 192]],

       [[192, 192, 192],
        [192, 192, 192],
        [192, 192, 192],
        ...,
        [192, 192, 192],
        [192, 192, 192],
        [192, 192, 192]],

       [[192, 192, 192],
        [192, 192, 192],
        [192, 192, 192],
        ...,
        [192, 192, 192],
        [192, 192, 192],
        [192, 192, 192]],

       ...,

       [[192, 192, 192],
        [192, 192, 192],
        [192, 192, 192],
        ...,
        [192, 192, 192],
        [192, 192, 192],
        [193, 191, 194]],

       [[192, 192, 192],
        [192, 192, 192],
        [192, 192, 192],
        ...,
        [192, 192, 192],
        [192, 192, 192],
        [193, 191, 194]],

       [[192, 192, 192],
        [192, 192, 192],
        [192, 192, 192],
        ...,
        [192, 192, 192],
        [192, 192, 190],
        [193, 191, 192]]